In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.5 MB/s eta 0:00:00


In [ ]:
import os
import random
import torch
import numpy as np
import cv2
import timm
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, RandomCrop, RandomRotation
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from PIL import Image

In [ ]:
class ColorectalCancerDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = self.img_paths[idx]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)  # Convert the NumPy array to a PIL Image
        img = self.transform(img)
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.img_paths)

In [ ]:
import tensorflow_datasets as tfds
dataset, info = tfds.load('colorectal_histology', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Shuffling /root/tensorflow_datasets/colorectal_histology/2.0.0.incompleteDOJKY0/colorectal_histology-train.tfr…

Dataset colorectal_histology downloaded and prepared to /root/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.


In [ ]:
# Set seed for reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define data augmentation transformations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),  # Add this line
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),  # Add this line
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Get the list of classes
classes = info.features['label'].names

# Get the image paths and corresponding labels
img_paths, labels = [], []
for example in dataset['train']:
    img_paths.append(example['image'].numpy())
    labels.append(example['label'].numpy())

# Split the data into train, validation, and test sets (70% train, 15% validation, 15% test)
train_img_paths, temp_img_paths, train_labels, temp_labels = train_test_split(
    img_paths, labels, test_size=0.3, stratify=labels, random_state=42
)
val_img_paths, test_img_paths, val_labels, test_labels = train_test_split(
    temp_img_paths, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)

train_dataset = ColorectalCancerDataset(train_img_paths, train_labels, transform=train_transform)
val_dataset = ColorectalCancerDataset(val_img_paths, val_labels, transform=test_transform)
test_dataset = ColorectalCancerDataset(test_img_paths, test_labels, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

xception = timm.create_model('xception', pretrained=True)
num_classes = len(set(train_labels))
xception.fc = nn.Linear(xception.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xception.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(xception.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)

num_epochs = 50

accuracies = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Train the model
    xception.train()
    train_loss = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = xception(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f"Train Loss: {train_loss}")

    # Validate the model
    xception.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = xception(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = correct / total
    accuracies.append(val_acc)
    if val_acc >= max(accuracies):
        torch.save(xception.state_dict(), "best_model.pt")
    print(f"Val Loss: {val_loss}, Val Acc: {val_acc}")

print("Average accuracy: ", sum(accuracies) / len(accuracies))


# Test the model
xception.load_state_dict(torch.load("best_model.pt"))  # Load the best model
xception.eval()

all_outputs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = xception(inputs)
        _, predicted = outputs.max(1)
        all_outputs.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_outputs)
test_precision = precision_score(all_labels, all_outputs, average='weighted')
test_recall = recall_score(all_labels, all_outputs, average='weighted')
test_f1 = f1_score(all_labels, all_outputs, average='weighted')

print("Test Accuracy: {:.4f}".format(test_accuracy))
print("Test Precision: {:.4f}".format(test_precision))
print("Test Recall: {:.4f}".format(test_recall))
print("Test F1-score: {:.4f}".format(test_f1))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


Epoch 1/50


100%|██████████| 110/110 [00:36<00:00,  3.01it/s]


Train Loss: 1.6171577843752774
Val Loss: 1.1482722014188766, Val Acc: 0.8266666666666667
Epoch 2/50


100%|██████████| 110/110 [00:29<00:00,  3.76it/s]


...

Train Loss: 0.016395499550906772
Val Loss: 0.09255047138625135, Val Acc: 0.9613333333333334
Epoch 49/50


100%|██████████| 110/110 [00:28<00:00,  3.86it/s]


Train Loss: 0.02609866404884749
Val Loss: 0.1065104897134006, Val Acc: 0.9613333333333334
Epoch 50/50


100%|██████████| 110/110 [00:28<00:00,  3.84it/s]


Train Loss: 0.016767633334331383
Val Loss: 0.10384281626708496, Val Acc: 0.9626666666666667
Average accuracy:  0.9474933333333333
Test Accuracy: 0.9440
Test Precision: 0.9460
Test Recall: 0.9440
Test F1-score: 0.9446
